In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [2]:
train_file = '/data/sulixin/research/contest/pytorch-pretrained-BERT/data/datasets/squadv2/train-v2.0.json'
dev_file = '/data/sulixin/research/contest/pytorch-pretrained-BERT/data/datasets/squadv2/dev-v2.0.json'

In [3]:
train = json.load(open(train_file))
dev = json.load(open(dev_file))

In [4]:
train['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [20]:
train['data'][23]['paragraphs'][0]

{'qas': [{'question': 'What is the French name of the Canadian Armed Forces?',
   'id': '56dde02666d3e219004dad6b',
   'answers': [{'text': 'Forces armées canadiennes', 'answer_start': 40}],
   'is_impossible': False},
  {'question': 'What law constitutes the CAF?',
   'id': '56dde02666d3e219004dad6c',
   'answers': [{'text': 'the National Defence Act', 'answer_start': 191}],
   'is_impossible': False},
  {'question': 'Who are the CAF supposed to protect?',
   'id': '56dde02666d3e219004dad6d',
   'answers': [{'text': 'Her Majesty', 'answer_start': 276}],
   'is_impossible': False},
  {'question': 'What does CF stand for in this context?',
   'id': '56dde02666d3e219004dad6e',
   'answers': [{'text': 'Forces canadiennes', 'answer_start': 110}],
   'is_impossible': False},
  {'plausible_answers': [{'text': 'Canadian Armed Forces', 'answer_start': 4}],
   'question': 'What does CFA stand for?',
   'id': '5ad3714d604f3c001a3fe285',
   'answers': [],
   'is_impossible': True},
  {'plausible_

In [24]:
def iter_para(data):
    for w in data:
        for p in w['paragraphs']:
            yield p

def iter_qa(data):
    for w in data:
        for p in w['paragraphs']:
            for qa in p['qas']:
                yield qa

            
def word_len(x):
    return len(x.split())

def avg(li):
    if len(li) == 0:
        return None
    return sum(li) * 1.0 / len(li)

def avg_query_len(c):
    return avg( [word_len(qa['question']) for qa in c['qas'] ] )

def qa_cnt(c):
    return len(c['qas'])
    
def my_stat(data):
    rv = {'para_len': [word_len(x['context']) for x in iter_para(data)],\
          'ques_len': [avg_query_len(x) for x in iter_para(data)],\
          'qa_cnt': [qa_cnt(x) for x in iter_para(data)],\
         }
    return pd.DataFrame(rv).describe()

def my_stat_query(data):
    rv = {'ques_len1': [word_len(qa['question']) for qa in iter_qa(data)],\
          'ans_len': [(word_len(qa['answers'][0]['text']) if not qa['is_impossible'] else 0) for qa in iter_qa(data)],\
          'answerable': [int(not qa['is_impossible']) for qa in iter_qa(data)]
    }
    return pd.DataFrame(rv).describe()


In [25]:
my_stat(train['data'])

,para_len,ques_len,qa_cnt
count,19035.000000,19035.000000,19035.000000
mean,116.585500,9.953592,6.846283
std,49.670155,2.266163,3.090609
min,20.000000,4.000000,1.000000
25%,87.000000,8.400000,5.000000
50%,107.000000,9.666667,5.000000
75%,139.000000,11.200000,9.000000
max,653.000000,29.000000,30.000000


In [26]:
my_stat_query(train['data'])

,ques_len1,ans_len,answerable
count,130319.000000,130319.000000,130319.000000
mean,9.893822,2.107099,0.666219
std,3.418829,3.144031,0.471564
min,1.000000,0.000000,0.000000
25%,7.000000,0.000000,0.000000
50%,9.000000,1.000000,1.000000
75%,12.000000,3.000000,1.000000
max,40.000000,43.000000,1.000000
